# Pandas + Clickhouse
1. Ставим Docker desktop
2. Устанавливаем [образ](https://hub.docker.com/r/yandex/clickhouse-server/) Clickhouse
```
!docker run -d -p 0.0.0.0:8123:8123 --volume=/path/to/some/folder/on/disk/some_clickhouse_database:/var/lib/clickhouse --name some-clickhouse-server --ulimit nofile=262144:262144 yandex/clickhouse-server
```

Опция -p 0.0.0.0:8123:8123 открывает доступ к контейнеру по порту 8123 (иногда сразу его нет). В параметре volume пропишите папку на своем компьютере, где база будет хранить данные.


3. Проверьте наличие доступа к clickhouse в контейнере в браузере, открыв ссылку [localhost:8123](http://localhost:8123), должны увидеть Ok.

4. Берем открытые данные [Метрики](https://clickhouse.tech/docs/ru/getting-started/example-datasets/metrica/).

In [ ]:
!docker pull postgres

In [1]:
import clickhouse_connect

In [2]:
client = clickhouse_connect.get_client(host='localhost', username='', password='')

In [3]:
client.command('select count(*) from datasets.hits')

53243388

In [4]:
# размер данных

client.query_df("""
    SELECT 
        table,
        round(sum(data_uncompressed_bytes) / 1024 / 1024 / 1024, 1) as uncompressed_gb,
        round(sum(data_compressed_bytes) / 1024 / 1024 / 1024, 1) as compressed_gb,
        round(uncompressed_gb / compressed_gb, 1) as compression_rate

    FROM system.parts
    WHERE table = 'hits'
    GROUP BY table
""")

,table,uncompressed_gb,compressed_gb,compression_rate
0,hits,34.4,2.9,11.9


In [5]:
%%time

client.query_df('select EventDate, count(*) as hits from datasets.hits group by EventDate order by EventDate')

CPU times: user 3.27 ms, sys: 3.85 ms, total: 7.12 ms
Wall time: 199 ms


,EventDate,hits
0,2014-03-17,8441748
1,2014-03-18,8301948
2,2014-03-19,8434782
3,2014-03-20,8121738
4,2014-03-21,7474674
5,2014-03-22,6189552
6,2014-03-23,6278946


In [6]:
%%time

df = client.query_df('select TraficSourceID, EventDate, uniqExact(UserID) from datasets.hits group by TraficSourceID, EventDate')


CPU times: user 81.9 ms, sys: 3.37 ms, total: 85.3 ms
Wall time: 678 ms


In [7]:
df

,TraficSourceID,EventDate,uniqExact(UserID)
0,4,2014-03-21,528
1,6,2014-03-20,1294
2,2,2014-03-18,1606
3,-1,2014-03-21,21386
4,10,2014-03-21,3824
...,...,...,...
65,-1,2014-03-23,18574
66,10,2014-03-23,3434
67,6,2014-03-17,1410
68,0,2014-03-22,26854
